In [1]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

### IMDB 리뷰 데이터
- sentence-piece v.s. word-piece 비교

### 1) Sentence-piece

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x1f97bc9e9a0>)

In [5]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [6]:
# 총 50000개 샘플 존재
print('리뷰 개수: ', len(train_df))

리뷰 개수:  50000


In [7]:
# sentence-piece의 입력으로 사용하기 위해 df를 txt로 저장
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [8]:
# sentence-piece로 각 단어에 고유한 정수 부여 (vocab 생성 과정)
### vocab size 5000으로 지정
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

#### -> vocab 생성이 완료되면, imdb.model, imdb.vocab 파일 2개가 생성된

In [9]:
# vocab 파일을 df로 저장
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
4485,▁unnecessary,-4482
4235,apped,-4232
2821,▁note,-2818
3561,IT,-3558
1127,▁comm,-1124
1232,▁song,-1229
2189,▁sequel,-2186
1788,ances,-1785
1192,▁disappoint,-1189
2509,Ch,-2506


In [10]:
len(vocab_list)

5000

In [12]:
# model 파일을 로드하여 인코딩 및 디코딩 작업 수행 가능
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

인코딩
- encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환
- encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환

In [14]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



### 2. Word-piece
- BertWordPieceTokenizer로 실습
- 네이버 영화 리뷰 데이터 NSMC data 이용

In [16]:
from tokenizers import BertWordPieceTokenizer

In [15]:
# NSMC data load
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x1f901253400>)

In [20]:
# txt 파일을 df로 로드 후, 결측값 제거
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')

In [21]:
# word-piece의 입력으로 사용하기 위해 df를 txt로 저장
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [24]:
# BertWordPiece Tokenizer 설정
word_tokenizer = BertWordPieceTokenizer(lowercase=False)

In [25]:
# NSMC 데이터 학습하여 단어 집합 얻기
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

word_tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency)

In [27]:
# vocab 저장
word_tokenizer.save_model('./')

['./vocab.txt']

In [29]:
# vocab을 df로 로드
### 앞서 vocab_size 30000으로 만들었기 때문에 vocab 안에 30000개 단어 존재
df = pd.read_fwf('vocab.txt', header = None)
df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,말과
29996,말들이
29997,말라는
29998,말밖에는


In [31]:
# Word-piece 방식
encoded = word_tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',word_tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20631, 3372, 3327, 24681, 7873, 7378]
디코딩 : 아 배고픈데 짜장면먹고싶다


In [41]:
# Word-piece 방식
encoded2 = word_tokenizer.encode('아름다운 대한민국 금수강산')
print('토큰화 결과 :',encoded2.tokens)
print('정수 인코딩 :',encoded2.ids)
print('디코딩 :',word_tokenizer.decode(encoded2.ids))

토큰화 결과 : ['아름다운', '대한민국', '금', '##수', '##강', '##산']
정수 인코딩 : [6225, 7117, 720, 3257, 3727, 3788]
디코딩 : 아름다운 대한민국 금수강산


### 3) NSMC로 sentence-piece 학습시키기 (비교 위해)

In [34]:
# sentence-piece로 각 단어에 고유한 정수 부여 (vocab 생성 과정)
### vocab size 5000으로 지정
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=rating --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

#### -> vocab 생성이 완료되면, rating.model, rating.vocab 파일 2개가 생성된

In [36]:
# vocab 파일을 df로 저장
vocab_list = pd.read_csv('rating.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
291,▁로,-288
3875,낫,-3872
1630,내가,-1627
3094,▁분들,-3091
1307,▁알게,-1304
3789,락,-3786
1573,▁봄,-1570
497,▁8,-494
2292,or,-2289
326,ᄏᄏᄏ,-323


In [37]:
len(vocab_list)

5000

In [38]:
# model 파일을 로드하여 인코딩 및 디코딩 작업 수행 가능
sp = spm.SentencePieceProcessor()
vocab_file_NSMC = "rating.model"
sp.load(vocab_file_NSMC)

True

In [40]:
# 인코딩 using sentence-piece 방식
lines = [
  "아 배고픈데 짜장면먹고싶다",
  "아름다운 대한민국 금수강산"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

아 배고픈데 짜장면먹고싶다
['▁아', '▁배', '고', '픈', '데', '▁짜', '장면', '먹고', '싶다']
[7, 81, 3280, 3726, 3316, 273, 557, 2607, 750]

아름다운 대한민국 금수강산
['▁아름다운', '▁대한민국', '▁금', '수', '강', '산']
[694, 1862, 1712, 3335, 3529, 3645]

